In [2]:
import xgboost as xgb
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import os
os.listdir('./data')

['train.csv', 'test.csv']

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train_id = train['id']
train_label = train['target']
del train['id'], train['target']

test_id = test['id']
del test['id']

data = train.append(test)
data.reset_index(inplace = True)

train_rows = train.shape[0]

- 변수들의 이름을 보면, 'car','ind','reg'로 각 대분류 카테고리가 정해져 있다
- 각 대분류 카테고리에 대하여 나머지 두 개의 대분류 벼수를 기반으로 운전자별 변수 값을 예측(?)
- 의문이 드는게 있는 변수들을 이용해서 예측을 하여 새로운 변수들을 만든다? 의미가 있나?
- 특히 선형모델을 이용할 경우 정말 의미가 없을듯
- 결과론적으로 얘기하면, 에측하련느 변수를 기준으로 다른 변수들의 분포에 대한 정보가 새롭게 담김

In [4]:
feature_results = []

In [10]:
eta = 0.1
max_depth = 6
subsample = 0.9
colsample_bytree = 0.85
min_child_weight = 55
num_boost_round = 500

params = {
        'objective' : 'reg:linear',
        'eta' : eta,
        'max_depth' : max_depth,
        'subsample' : subsample,
        'colsample_bytree' : colsample_bytree,
        'min_child_weight' : min_child_weight,
        'silent' : 1,
        'tree_method' : 'gpu_hist'
}

In [23]:
for target_g in ['car','ind','reg'] :
    features = [x for x in list(data) if target_g not in x] #list(data) = data.columns
    target_list = [x for x in list(data) if target_g in x]
    
    train_fea = np.array(data[features])
    for target in target_list :
        print(target)
        train_label = data[target]
        
        kfold = KFold(n_splits = 5, random_state = 218, shuffle = True)
        kf = kfold.split(data)
        cv_train = np.zeros(shape = (len(data),1))
        
        for i, (train_fold, valid_fold) in enumerate(kf) :
            
            X_train, X_valid = train_fea[train_fold, :], train_fea[valid_fold, :]
            y_train, y_valid = train_label[train_fold], train_label[valid_fold]
            
            dtrain = xgb.DMatrix(X_train, y_train)
            dvalid = xgb.DMatrix(X_valid, y_valid)
            
            watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
            
            bst = xgb.train(params, dtrain, num_boost_round, evals = watchlist, verbose_eval=50, early_stopping_rounds=10)
            
            cv_train[valid_fold, 0] += bst.predict(xgb.DMatrix(X_valid), ntree_limit = bst.best_ntree_limit)
        
        feature_results.append(cv_train)

ps_car_01_cat
[0]	train-rmse:7.4387	valid-rmse:7.44276
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.30729	valid-rmse:2.31257
[100]	train-rmse:2.29756	valid-rmse:2.30508
[150]	train-rmse:2.2928	valid-rmse:2.30306
[200]	train-rmse:2.28868	valid-rmse:2.30155
[250]	train-rmse:2.28484	valid-rmse:2.30039
[300]	train-rmse:2.28155	valid-rmse:2.29992
Stopping. Best iteration:
[333]	train-rmse:2.2793	valid-rmse:2.29953

[0]	train-rmse:7.43978	valid-rmse:7.43787
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:2.30744	valid-rmse:2.31025
[100]	train-rmse:2.29778	valid-rmse:2.30312
[150]	train-rmse:2.29329	valid-rmse:2.30121
[200]	train-rmse:2.28899	valid-rmse:2.29976
[250]	train-rmse:2.28562	valid-rmse:2.29921
[300]	train-rmse:2.28216	valid-rmse:2.29857
Stopping. Best ite

Stopping. Best iteration:
[183]	train-rmse:1.97618	valid-rmse:1.97005

[0]	train-rmse:2.13844	valid-rmse:2.13741
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.98558	valid-rmse:1.98977
[100]	train-rmse:1.97875	valid-rmse:1.98649
Stopping. Best iteration:
[136]	train-rmse:1.97556	valid-rmse:1.98597

ps_car_05_cat
[0]	train-rmse:1.02413	valid-rmse:1.02524
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.769072	valid-rmse:0.77002
[100]	train-rmse:0.765649	valid-rmse:0.767129
[150]	train-rmse:0.76423	valid-rmse:0.766551
[200]	train-rmse:0.76311	valid-rmse:0.766305
[250]	train-rmse:0.761963	valid-rmse:0.766092
[300]	train-rmse:0.760975	valid-rmse:0.766025
Stopping. Best iteration:
[290]	train-rmse:0.76116	valid-rmse:0.766019

[0]	train-rmse:1.0241	valid-rmse:1.0253

Stopping. Best iteration:
[96]	train-rmse:0.348371	valid-rmse:0.349405

ps_car_09_cat
[0]	train-rmse:1.22494	valid-rmse:1.22513
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.91849	valid-rmse:0.916961
[100]	train-rmse:0.914257	valid-rmse:0.913628
[150]	train-rmse:0.912413	valid-rmse:0.912835
[200]	train-rmse:0.910981	valid-rmse:0.912551
Stopping. Best iteration:
[215]	train-rmse:0.91057	valid-rmse:0.912475

[0]	train-rmse:1.22454	valid-rmse:1.22696
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.918318	valid-rmse:0.918068
[100]	train-rmse:0.91414	valid-rmse:0.914675
[150]	train-rmse:0.912466	valid-rmse:0.913944
[200]	train-rmse:0.91096	valid-rmse:0.913515
Stopping. Best iteration:
[227]	train-rmse:0.910317	valid-rmse:0.913367

[0]	train-rmse:1.22483	valid-rms

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.052167	valid-rmse:0.05245
[100]	train-rmse:0.051986	valid-rmse:0.052329
[150]	train-rmse:0.051897	valid-rmse:0.052306
Stopping. Best iteration:
[160]	train-rmse:0.051881	valid-rmse:0.052302

[0]	train-rmse:0.122356	valid-rmse:0.122386
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.05223	valid-rmse:0.052182
[100]	train-rmse:0.052057	valid-rmse:0.05207
[150]	train-rmse:0.051956	valid-rmse:0.052039
Stopping. Best iteration:
[177]	train-rmse:0.051914	valid-rmse:0.052035

[0]	train-rmse:0.122337	valid-rmse:0.122461
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.052149	valid-rmse:0.052518
[100]	train-rmse:0.051973	valid-rmse:0.052399
[150]	train-rmse:0.051883	valid-rmse:0.052373
Stopping. 

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:238375	valid-rmse:238873
Stopping. Best iteration:
[53]	train-rmse:238349	valid-rmse:238872

ps_ind_01
[0]	train-rmse:2.32677	valid-rmse:2.32712
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.73933	valid-rmse:1.74222
[100]	train-rmse:1.7281	valid-rmse:1.73359
[150]	train-rmse:1.7221	valid-rmse:1.72999
[200]	train-rmse:1.71822	valid-rmse:1.72827
[250]	train-rmse:1.71504	valid-rmse:1.72744
[300]	train-rmse:1.71211	valid-rmse:1.7268
[350]	train-rmse:1.7094	valid-rmse:1.72636
[400]	train-rmse:1.70688	valid-rmse:1.7261
Stopping. Best iteration:
[439]	train-rmse:1.70481	valid-rmse:1.72579

[0]	train-rmse:2.32736	valid-rmse:2.3239
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:1.73978	valid-rms

...

[50]	train-rmse:0.385213	valid-rmse:0.38543
[100]	train-rmse:0.381218	valid-rmse:0.381887
[150]	train-rmse:0.379286	valid-rmse:0.380387
[200]	train-rmse:0.377911	valid-rmse:0.379492
[250]	train-rmse:0.376999	valid-rmse:0.37907
[300]	train-rmse:0.376137	valid-rmse:0.378634
[350]	train-rmse:0.375412	valid-rmse:0.378379
[400]	train-rmse:0.374763	valid-rmse:0.378189
[450]	train-rmse:0.374145	valid-rmse:0.378019
[499]	train-rmse:0.373542	valid-rmse:0.377884
[0]	train-rmse:0.483722	valid-rmse:0.483699
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.385641	valid-rmse:0.385464
[100]	train-rmse:0.381371	valid-rmse:0.381536
[150]	train-rmse:0.379273	valid-rmse:0.379876
[200]	train-rmse:0.378079	valid-rmse:0.37916
[250]	train-rmse:0.377164	valid-rmse:0.378683
[300]	train-rmse:0.376183	valid-rmse:0.378178
[350]	train-rmse:0.375446	valid-rmse:0.377886
[400]	train-rmse:0.374715	valid-rm

[499]	train-rmse:0.333754	valid-rmse:0.338554
[0]	train-rmse:0.477933	valid-rmse:0.477903
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.344124	valid-rmse:0.344164
[100]	train-rmse:0.340425	valid-rmse:0.340846
[150]	train-rmse:0.3389	valid-rmse:0.339725
[200]	train-rmse:0.337884	valid-rmse:0.339105
[250]	train-rmse:0.337029	valid-rmse:0.338631
[300]	train-rmse:0.33633	valid-rmse:0.338327
[350]	train-rmse:0.335699	valid-rmse:0.338121
[400]	train-rmse:0.33516	valid-rmse:0.337949
[450]	train-rmse:0.334618	valid-rmse:0.337819
[499]	train-rmse:0.334112	valid-rmse:0.337705
ps_ind_10_bin
[0]	train-rmse:0.450081	valid-rmse:0.450078
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.019471	valid-rmse:0.018838
Stopping. Best iteration:
[69]	train-rmse:0.0193	valid-rmse:0.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.125474	valid-rmse:0.12442
Stopping. Best iteration:
[61]	train-rmse:0.125387	valid-rmse:0.124403

ps_ind_15
[0]	train-rmse:7.06226	valid-rmse:7.07381
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:3.44939	valid-rmse:3.45114
[100]	train-rmse:3.43976	valid-rmse:3.44604
[150]	train-rmse:3.43365	valid-rmse:3.44438
[200]	train-rmse:3.42864	valid-rmse:3.44382
[250]	train-rmse:3.42425	valid-rmse:3.44347
Stopping. Best iteration:
[255]	train-rmse:3.42377	valid-rmse:3.44343

[0]	train-rmse:7.06339	valid-rmse:7.06894
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:3.44857	valid-rmse:3.45531
[100]	train-rmse:3.43934	valid-rmse:3.45028
[150]	train-rmse:3.43347	valid-rmse:3.44864
[200]	train-rmse:3.42

[100]	train-rmse:0.355084	valid-rmse:0.356516
[150]	train-rmse:0.354581	valid-rmse:0.356458
Stopping. Best iteration:
[151]	train-rmse:0.354568	valid-rmse:0.356455

ps_reg_01
[0]	train-rmse:0.301916	valid-rmse:0.301684
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.259533	valid-rmse:0.259733
[100]	train-rmse:0.256413	valid-rmse:0.256943
[150]	train-rmse:0.255235	valid-rmse:0.256077
[200]	train-rmse:0.254501	valid-rmse:0.255654
[250]	train-rmse:0.253921	valid-rmse:0.255395
[300]	train-rmse:0.253447	valid-rmse:0.255228
[350]	train-rmse:0.253023	valid-rmse:0.25512
[400]	train-rmse:0.252646	valid-rmse:0.255059
[450]	train-rmse:0.252277	valid-rmse:0.254997
Stopping. Best iteration:
[459]	train-rmse:0.252218	valid-rmse:0.254987

[0]	train-rmse:0.301883	valid-rmse:0.301805
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until val

[0]	train-rmse:0.788976	valid-rmse:0.787459
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.72152	valid-rmse:0.720558
[100]	train-rmse:0.716136	valid-rmse:0.71612
[150]	train-rmse:0.713612	valid-rmse:0.714514
[200]	train-rmse:0.711897	valid-rmse:0.713791
[250]	train-rmse:0.710509	valid-rmse:0.713407
[300]	train-rmse:0.709231	valid-rmse:0.713095
[350]	train-rmse:0.708141	valid-rmse:0.712916
[400]	train-rmse:0.70704	valid-rmse:0.712791
Stopping. Best iteration:
[398]	train-rmse:0.707076	valid-rmse:0.712781

[0]	train-rmse:0.788518	valid-rmse:0.788978
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[50]	train-rmse:0.721224	valid-rmse:0.722846
[100]	train-rmse:0.715338	valid-rmse:0.717897
[150]	train-rmse:0.712831	valid-rmse:0.716357
[200]	train-rmse:0.71118	valid-rmse:0.715664
[25

In [31]:
len(feature_results), len(feature_results[0])

(38, 1488028)

In [33]:
feature_results = np.hstack(feature_results)

train_features = feature_results[:train_rows, :]
test_features = feature_results[train_rows:,:]

In [34]:
import pickle
#save
pickle.dump([train_features, test_features], open('./data/fea0.pk','wb'))